# Feature Selection

1. Remove irrelevant features results in a better performance model
2. in a
3. in a model that run faster

In [1]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
%%time
X_train = pd.read_csv('../data/interim/X_train.csv')
y_train = pd.read_csv('../data/interim/y_train.csv')
X_test = pd.read_csv('../data/interim/X_test.csv')
y_test = pd.read_csv('../data/interim/y_test.csv')

Wall time: 14.1 s


In [3]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3204902 entries, 0 to 3204901
Data columns (total 44 columns):
 #   Column                           Dtype  
---  ------                           -----  
 0   problem_number                   int64  
 1   exercise_problem_repeat_session  int64  
 2   is_correct                       bool   
 3   total_sec_taken                  int64  
 4   total_attempt_cnt                int64  
 5   used_hint_cnt                    int64  
 6   is_hint_used                     bool   
 7   is_downgrade                     int64  
 8   is_upgrade                       int64  
 9   level                            int64  
 10  proficient_correct_rate          float64
 11  problem_correct_rate             float64
 12  points                           int64  
 13  badges_cnt                       int64  
 14  user_grade                       int64  
 15  has_teacher_cnt                  int64  
 16  is_self_coach                    bool   
 17  has_stud

### Amount of variation

The variable has to be reviewed and dropped if the variable has a very low variation, because the model is not going to learn anything with it.

### Pairwise correlations

a

In [4]:
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier as rf

selector = SelectFromModel(rf(n_estimators=100, random_state=0))
selector.fit(X_train, y_train.values.ravel())
support = selector.get_support()
features = X_train.loc[:,support].columns.tolist()
print(features)
print(rf(n_estimators=100, random_state=0).fit(X_train,y_train).feature_importances_)

['problem_number', 'is_upgrade', 'level', 'proficient_correct_rate']


C:\Users\likwe\anaconda3\lib\site-packages\ipykernel_launcher.py:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  if __name__ == '__main__':


[4.18104547e-02 8.55832950e-03 6.77943640e-04 3.36769235e-03
 4.63229359e-04 7.66200486e-04 9.76799639e-04 7.36851254e-06
 2.53438072e-02 7.61724048e-01 1.29342772e-01 6.55159603e-03
 3.38465727e-03 2.70496276e-03 1.35374569e-03 9.61131726e-04
 1.01187243e-04 3.29619976e-04 1.04457743e-03 4.22587382e-04
 3.17209157e-04 5.65897116e-04 1.36735621e-04 1.15107595e-04
 1.41645065e-04 9.57194378e-05 1.25109128e-04 1.44850717e-04
 4.52228676e-05 5.59094639e-05 2.25600011e-10 1.96287184e-04
 1.28099123e-04 2.94726539e-04 1.36460887e-05 2.03289855e-04
 2.00095539e-04 1.71929977e-04 2.60031887e-04 1.22811379e-04
 2.25454679e-04 6.41366373e-04 4.91962730e-04 5.41418141e-03]


### PCA

Principal Component Analysis, or PCA, is a dimensionality-reduction method that is often used to reduce the dimensionality of large data sets, by transforming a large set of variables into a smaller one that still contains most of the information in the large set.

In [5]:
# Applying PCA
from sklearn.decomposition import PCA
pca = PCA(n_components = 2)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

In [6]:
# Training the Logistic Regression model on the Training set
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
classifier.fit(X_train, y_train)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

C:\Users\likwe\anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[[694783 359023]
 [  6592   7903]]


0.6577603128706235

In [8]:
# Training the Random Forest Classification model on the Training set
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
classifier.fit(X_train, y_train)

C:\Users\likwe\anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [10]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)
#print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1051620    2186]
 [  13138    1357]]


0.9856557281140802

In [15]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

# Visualising the Training set results
from matplotlib.colors import ListedColormap
X_set, y_set = sc.inverse_transform(X_train), y_train
X1, X2 = np.meshgrid(np.arange(start = X_set[:, 0].min() - 10, stop = X_set[:, 0].max() + 10, step = 0.25),
                     np.arange(start = X_set[:, 1].min() - 1000, stop = X_set[:, 1].max() + 1000, step = 0.25))
plt.contourf(X1, X2, classifier.predict(sc.transform(np.array([X1.ravel(), X2.ravel()]).T)).reshape(X1.shape),
             alpha = 0.75, cmap = ListedColormap(('red', 'green')))
plt.xlim(X1.min(), X1.max())
plt.ylim(X2.min(), X2.max())
for i, j in enumerate(np.unique(y_set)):
    plt.scatter(X_set[y_set == j, 0], X_set[y_set == j, 1], c = ListedColormap(('red', 'green'))(i), label = j)
plt.title('Random Forest Classification (Training set)')
plt.xlabel('Age')
plt.ylabel('Estimated Salary')
plt.legend()
plt.show()

NotFittedError: This StandardScaler instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.